In [1]:
from google.cloud import bigquery
import pandas as pd
import sqlite3

Connect to BigQuery and fetch the necessary data:

In [ ]:
# Initialize a BigQuery client
client = bigquery.Client()

# List of transaction tables from umt-msba project (excluding inactive ones)
table_names = [
    'transArchive_201001_201003', 'transArchive_201004_201006', 'transArchive_201007_201009', 
    'transArchive_201010_201012', 'transArchive_201101_201103', 'transArchive_201104', 
    'transArchive_201105', 'transArchive_201106', 'transArchive_201107_201109', 
    'transArchive_201110_201112', 'transArchive_201201_201203', 'transArchive_201204_201206', 
    'transArchive_201207_201209', 'transArchive_201210_201212', 'transArchive_201301_201303', 
    'transArchive_201304_201306', 'transArchive_201307_201309', 'transArchive_201310_201312', 
    'transArchive_201401_201403', 'transArchive_201404_201406', 'transArchive_201407_201409', 
    'transArchive_201410_201412', 'transArchive_201501_201503', 'transArchive_201504_201506', 
    'transArchive_201507_201509', 'transArchive_201510', 'transArchive_201511', 'transArchive_201512', 
    'transArchive_201601', 'transArchive_201602', 'transArchive_201603', 'transArchive_201604', 
    'transArchive_201605', 'transArchive_201606', 'transArchive_201607', 'transArchive_201608', 
    'transArchive_201609', 'transArchive_201610', 'transArchive_201611', 'transArchive_201612', 
    'transArchive_201701'
]

# Construct a UNION query across all tables to gather data excluding non-owners (card_no != 3)
union_query = " UNION ALL ".join([
    f"SELECT datetime, card_no, department, description, upc, unitPrice, total, trans_status, "
    f"EXTRACT(YEAR FROM datetime) AS year, EXTRACT(MONTH FROM datetime) AS month, "
    f"EXTRACT(DAY FROM datetime) AS day, EXTRACT(HOUR FROM datetime) AS hour "
    f"FROM `umt-msba.transactions.{table_name}` WHERE card_no != 3" 
    for table_name in table_names
])

# Add the department_lookup table to join and fetch department names
full_query = f"""
    WITH owner_transactions AS (
        {union_query}
    )
    SELECT ot.*, dl.dept_name
    FROM owner_transactions ot
    LEFT JOIN `umt-msba.transactions.department_lookup` dl
    ON ot.department = dl.department
"""
# ADJUST LIMIT WHEN RUNNING FOR REAL

# Execute the query and fetch the data
df = client.query(full_query).to_dataframe()

# Display the DataFrame
print(df.head())


In [7]:
# Initialize a BigQuery client
client = bigquery.Client()

# Load the sample data from the CSV file
df = pd.read_csv('G:/College/University of Montana/Semester 3/Applied Data Analytics/Wedge Project/Wedge-Project-ADA-Riley-ORorke/owner_transactions.csv', encoding='ISO-8859-1')

# Query the department_lookup table from BigQuery
lookup_query = """
    SELECT department, dept_name
    FROM `umt-msba.transactions.department_lookup`
"""
# Execute the query and fetch the department_lookup data
department_lookup_df = client.query(lookup_query).to_dataframe()

# Join the CSV data with the department_lookup data
df = df.merge(department_lookup_df, on='department', how='left')

# Display the merged DataFrame
print(df.head())



C:\Users\Riley_26\AppData\Local\Temp\ipykernel_24268\1436812371.py:5: DtypeWarning: Columns (33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('G:/College/University of Montana/Semester 3/Applied Data Analytics/Wedge Project/Wedge-Project-ADA-Riley-ORorke/owner_transactions.csv', encoding='ISO-8859-1')
C:\Users\Riley_26\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Data with Department Names:
                    datetime  register_no  emp_no  trans_no  upc description  \
0  2016-09-01 11:28:18+00:00         51.0    94.0     141.0  TAX         Tax   
1  2016-09-01 11:39:03+00:00         51.0    94.0     150.0  TAX         Tax   
2  2016-09-01 11:13:19+00:00          1.0    39.0      13.0  TAX         Tax   
3  2016-09-01 11:41:29+00:00          4.0    36.0      58.0  TAX         Tax   
4  2016-09-01 16:48:30+00:00          2.0    26.0      70.0  TAX         Tax   

  trans_type trans_subtype trans_status  department  ...  local  organic  \
0          A           NaN          NaN         0.0  ...    0.0      NaN   
1          A           NaN          NaN         0.0  ...    0.0      NaN   
2          A           NaN          NaN         0.0  ...    0.0      NaN   
3          A           NaN          NaN         0.0  ...    0.0      NaN   
4          A           NaN          NaN         0.0  ...    0.0      NaN   

   display  receipt  card_no  stor

Build Summary Tables  
    
      
      
      
1. Sales by date by hour:

In [9]:
# Ensure 'datetime' column is in datetime format
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')

# Create 'calendar_date' column from the 'datetime' column
df['calendar_date'] = df['datetime'].dt.date  # Extract date part (YYYY-MM-DD)
df['hour'] = df['datetime'].dt.hour  # Extract hour part

# Create the 'Sales by date by hour' summary table
sales_by_hour = df.groupby(['calendar_date', 'hour']).agg(
    total_spend=('total', 'sum'),
    num_transactions=('datetime', 'count'),  # Use 'datetime' to count transactions
    num_items=('trans_status', lambda x: ((x == ' ') | (x == '')).sum())  # Count items where trans_status is blank or a single space
).reset_index()

# Display the result
print(sales_by_hour)



      calendar_date  hour   total_spend  num_transactions  num_items
0        2010-01-01    15  2.220446e-16                15         10
1        2010-01-01    16  0.000000e+00                13          9
2        2010-01-01    17  8.881784e-16                 6          2
3        2010-01-01    18 -1.221245e-14                88         73
4        2010-01-01    19  0.000000e+00                11          6
...             ...   ...           ...               ...        ...
23968    2017-01-30    21  0.000000e+00                 7          3
23969    2017-01-31    17  0.000000e+00                 8          4
23970    2017-01-31    19  1.776357e-15                11          3
23971    2017-01-31    20  1.110223e-15                20          7
23972    2017-01-31    21  4.267420e-16                 6          2

[23973 rows x 5 columns]


2. Sales by owner by year by month:

In [15]:
# Extract 'year' and 'month' from the 'datetime' column
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month

# Create the 'Sales by owner by year by month' summary table
sales_by_owner = df.groupby(['card_no', 'year', 'month']).agg(
    total_sales=('total', 'sum'),
    num_transactions=('trans_status', 'count'),
    num_items=('trans_status', lambda x: ((x == ' ') | (x == '')).sum())  # Correct logical OR operation
).reset_index()

# Display the result
print(sales_by_owner)


      card_no  year  month   total_sales  num_transactions  num_items
0     10067.0  2010      1  0.000000e+00                28         24
1     10067.0  2010      2  8.881784e-15                23         23
2     10067.0  2010      3 -3.774758e-15                17         17
3     10067.0  2010      4  0.000000e+00                23         23
4     10067.0  2010      5  2.664535e-15                43         42
...       ...   ...    ...           ...               ...        ...
9796  51049.0  2010      2  0.000000e+00                60         60
9797  51049.0  2010      3  2.664535e-15                11         11
9798  51049.0  2010      7  0.000000e+00                20         20
9799  51049.0  2011      4  1.287859e-14                42         42
9800  51171.0  2010      1  6.550316e-15                57         50

[9801 rows x 6 columns]


3. Sales by product description by year by month:

In [12]:
# Create the 'Sales by product description by year by month' summary table
sales_by_product = df.groupby(['upc', 'description', 'department', 'dept_name', 'year', 'month']).agg(
    total_sales=('total', 'sum'),
    num_transactions=('datetime', 'count'),  # Count transactions using 'datetime'
    num_items=('trans_status', lambda x: ((x == ' ') | (x == '')).sum())  # Correct logical OR operation to count items
).reset_index()

# Display the result
print(sales_by_product)


               upc                  description  department         dept_name  \
0                0                  BULK Coupon         3.0              BULK   
1                0                FROZEN Coupon         6.0            FROZEN   
2                0      PACKAGED GROCERY Coupon         1.0  PACKAGED GROCERY   
3                0      PACKAGED GROCERY Coupon         1.0  PACKAGED GROCERY   
4                0      PACKAGED GROCERY Coupon         1.0  PACKAGED GROCERY   
...            ...                          ...         ...               ...   
200821  WWDISCOUNT  Wellness Wednesday Discount        10.0       SUPPLEMENTS   
200822  WWDISCOUNT  Wellness Wednesday Discount        10.0       SUPPLEMENTS   
200823  WWDISCOUNT  Wellness Wednesday Discount        10.0       SUPPLEMENTS   
200824  WWDISCOUNT  Wellness Wednesday Discount        10.0       SUPPLEMENTS   
200825  WWDISCOUNT  Wellness Wednesday Discount        10.0       SUPPLEMENTS   

        year  month  total_

Create SQLite Database and Insert Data

In [16]:
# Create SQLite connection and cursor
conn = sqlite3.connect('wedge_coop_summary.db')
cursor = conn.cursor()

# Create the tables in SQLite
cursor.execute('''
CREATE TABLE sales_by_hour (
    year INTEGER,
    month INTEGER,
    day INTEGER,
    hour INTEGER,
    total_spend REAL,
    num_transactions INTEGER,
    num_items INTEGER
)''')

cursor.execute('''
CREATE TABLE sales_by_owner (
    card_no INTEGER,
    year INTEGER,
    month INTEGER,
    total_sales REAL,
    num_transactions INTEGER,
    num_items INTEGER
)''')

cursor.execute('''
CREATE TABLE sales_by_product (
    upc TEXT,
    description TEXT,
    department INTEGER,
    year INTEGER,
    month INTEGER,
    total_sales REAL,
    num_transactions INTEGER,
    num_items INTEGER
)''')

# Insert data into SQLite tables
# Insert sales by hour
sales_by_hour.to_sql('sales_by_hour', conn, if_exists='replace', index=False)

# Insert sales by owner
sales_by_owner.to_sql('sales_by_owner', conn, if_exists='replace', index=False)

# Insert sales by product
sales_by_product.to_sql('sales_by_product', conn, if_exists='replace', index=False)

# Commit changes and close the connection
conn.commit()
conn.close()

print("Summary tables created and stored in 'wedge_coop_summary.db'")


Summary tables created and stored in 'wedge_coop_summary.db'
